<a href="https://colab.research.google.com/github/imabari/musen/blob/main/musen_nara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

件数取得API/一覧取得APIのリクエスト条件一覧

https://www.tele.soumu.go.jp/resource/j/musen/webapi/mw_req_conditions.pdf


コード値一覧

https://www.tele.soumu.go.jp/resource/j/musen/webapi/mw_code.pdf


地方公共団体コード

http://www.soumu.go.jp/denshijiti/code.html

In [1]:
import io
import urllib.parse

import requests
import pandas as pd

# API

In [2]:
d = {
    # 1:免許情報検索  2: 登録情報検索
    "ST": 1,
    # 詳細情報付加 0:なし 1:あり
    "DA": 1,
    # スタートカウント
    "SC": 1,
    # 取得件数
    "DC": 1,
    # 出力形式 1:CSV 2:JSON 3:XML
    "OF": 1,
    # 無線局の種別
    "OW": "FB_H",
    # 所轄総合通信局
    "IT": "E",
    # 免許人名称/登録人名称
    "NA": "楽天モバイル",
}

parm = urllib.parse.urlencode(d, encoding="shift-jis")

In [3]:
r = requests.get("https://www.tele.soumu.go.jp/musen/list", parm)

# データラングリング

In [4]:
df0 = pd.read_csv(io.StringIO(r.text), skiprows=1, header=None, index_col=0)

In [5]:
df1 = df0[25].str.strip().str.split(r"\\n", 2, expand=True)

In [6]:
se = df1.loc[df1[0].str.contains("携帯電話（その他基地局等"), 2]

In [7]:
df2 = se.str.strip().str.replace(r"\\n", "").str.extractall("(.+?)\(([0-9,]+?)\)").rename(columns={0: "市区町村名", 1: "開設局数"}).reset_index(drop=True)

In [8]:
df2["市区町村名"] = df2["市区町村名"].str.strip()

In [9]:
df2["開設局数"] = df2["開設局数"].str.strip().str.replace(",", "").astype(int)

In [10]:
flag = df2["市区町村名"].str.endswith(("都","道","府","県"))

In [11]:
df2["都道府県名"] = df2["市区町村名"].where(flag).fillna(method="ffill")

In [12]:
df2

,市区町村名,開設局数,都道府県名
0,東京都,2619,東京都
1,港区,2619,東京都
2,神奈川県,3169,神奈川県
3,横浜市戸塚区,3169,神奈川県
4,滋賀県,164,滋賀県
...,...,...,...
158,御坊市,2,和歌山県
159,田辺市,3,和歌山県
160,日高郡みなべ町,1,和歌山県
161,西牟婁郡白浜町,6,和歌山県


# 都道府県

In [13]:
df_prefs = df2[flag].reset_index(drop=True)
df_prefs

,市区町村名,開設局数,都道府県名
0,東京都,2619,東京都
1,神奈川県,3169,神奈川県
2,滋賀県,164,滋賀県
3,京都府,375,京都府
4,大阪府,1518,大阪府
5,兵庫県,877,兵庫県
6,奈良県,187,奈良県
7,和歌山県,54,和歌山県


In [14]:
df_prefs.to_csv("prefs.csv", encoding="utf_8_sig")

# 市区町村

In [15]:
df_cities = df2[~flag].reset_index(drop=True).reindex(columns=["都道府県名", "市区町村名", "開設局数"])
df_cities

,都道府県名,市区町村名,開設局数
0,東京都,港区,2619
1,神奈川県,横浜市戸塚区,3169
2,滋賀県,大津市,39
3,滋賀県,彦根市,21
4,滋賀県,長浜市,1
...,...,...,...
150,和歌山県,御坊市,2
151,和歌山県,田辺市,3
152,和歌山県,日高郡みなべ町,1
153,和歌山県,西牟婁郡白浜町,6


In [16]:
df_cities.to_csv("cities.csv", encoding="utf_8_sig")

# 奈良県

In [17]:
df_nara = df_cities[df_cities["都道府県名"] == "奈良県"].set_index("市区町村名")

In [18]:
df_nara.drop("都道府県名", axis=1, inplace=True)

In [19]:
idx = (
    "奈良市",
    "大和高田市",
    "大和郡山市",
    "天理市",
    "橿原市",
    "桜井市",
    "五條市",
    "御所市",
    "生駒市",
    "香芝市",
    "葛城市",
    "宇陀市",
    "山添村",
    "平群町",
    "三郷町",
    "斑鳩町",
    "安堵町",
    "川西町",
    "三宅町",
    "田原本町",
    "曽爾村",
    "御杖村",
    "高取町",
    "明日香村",
    "上牧町",
    "王寺町",
    "広陵町",
    "河合町",
    "吉野町",
    "大淀町",
    "下市町",
    "黒滝村",
    "天川村",
    "野迫川村",
    "十津川村",
    "下北山村",
    "上北山村",
    "川上村",
    "東吉野村",
)

In [20]:
df_nara = df_nara.reindex(idx, fill_value=0)

In [21]:
df_nara["開設局数"].sum()

172

In [22]:
df_nara.to_csv("nara.csv", encoding="utf_8_sig")

# ダウンロード

In [23]:
from google.colab import files

In [24]:
files.download("nara.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>